<b> Organize the precipitation data into the respectice ENSO bins then save it as a netcdf file </b> 

In [1]:
import xarray as xr
from metpy.constants import g, omega as om, Re as a
from metpy.units import units
import netCDF4
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Upload HAD the seasonally average anomalies for the HAD 

dir = '/chinook/dallmann/Simulation/MPI/precip/'
fn_hadp_anom = 'pr_MPI_2020-2064_anomDJFmean.nc'


data_had_pr_his = xr.open_dataset(dir+fn_hadp_anom)

# Parse using MetPy, so we can use MetPy operations with it
#data_had_pr_his = data_had_pr_his.metpy.parse_cf()

# output directory and file
dro = '/chinook/dallmann/MPI/ELI_Bins/'
fn_output = 'fut_pr_nino_strong_MPI_anomDJFmean.nc'

In [3]:
had_newtime = data_had_pr_his
#had_newtime['time'] = data_had_pr_his.indexes['time'].to_datetimeindex(unsafe=True)

#had_newtime

In [4]:
# Upload the array for Nino_strong
dir = '/chinook/dallmann/ENSO_bins/'
fn_ENSO = 'MPI_future_nino_strong_DJFmean.nc'

data_ENSO = xr.open_dataset(dir+fn_ENSO)
data_ENSO

<xarray.Dataset>
Dimensions:  (year: 10)
Coordinates:
  * year     (year) int64 2020 2030 2031 2032 2038 2043 2049 2050 2054 2061
Data variables:
    eli      (year) float64 ...

In [5]:
# Convert time to year so it can be found in an Integer64
# I am unsure if this is right since it already was seasonally averaged (unsure what to do about the .mean())
time_had = had_newtime.groupby('time.year').mean()
# shift the temp data by one year because cdo reconizes the year assoicated with december. 
time_had = time_had.shift(year=1)

time_had

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 192, lon: 384, year: 46)
Coordinates:
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * year      (year) int64 2019 2020 2021 2022 2023 ... 2060 2061 2062 2063 2064
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (year, lon, bnds) float64 nan nan nan nan ... 358.6 358.6 359.5
    lat_bnds  (year, lat, bnds) float64 nan nan nan nan ... 88.82 88.82 89.75
    pr        (year, lat, lon) float32 nan nan nan ... 1.226e-06 1.176e-06

In [6]:
pr_nino_strong = time_had.where(time_had['year'] == data_ENSO['year'], drop = True)
pr_nino_strong

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 192, lon: 384, year: 10)
Coordinates:
  * year      (year) int64 2020 2030 2031 2032 2038 2043 2049 2050 2054 2061
  * lon       (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
  * lat       (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
Dimensions without coordinates: bnds
Data variables:
    lon_bnds  (year, lon, bnds) float64 -0.4688 0.4688 0.4688 ... 358.6 359.5
    lat_bnds  (year, lat, bnds) float64 -89.75 -88.82 -88.82 ... 88.82 89.75
    pr        (year, lat, lon) float32 1.054e-06 1.065e-06 ... 5.419e-07

In [7]:
# Create a new netcdf file for preciptiation relative to this respective year
pr_nino_strong.to_netcdf(path=dro+fn_output)